In [1]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
import mechanize

In [2]:
def obtain_url_researchgate(website_url):
    request = urllib.request.Request(
        url = website_url,
        data = None,
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
        }
    )

    testurl = urllib.request.urlopen(request)
    soup = BeautifulSoup(testurl, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))

In [3]:
obtain_url_researchgate("https://www.researchgate.net/publication/293012925_On_the_stability_of_an_Al-fumarate_MOF_and_its_potential_for_CO2_capture_from_wet_stream")

https://www.researchgate.net/profile/Juliana_Coelho5
https://www.researchgate.net/profile/Ana_Ribeiro25
https://www.researchgate.net/profile/Alexandre_Ferreira4
https://www.researchgate.net/scientific-contributions/2082644774_Sebastiao_MP_Lucena


In [4]:
obtain_url_researchgate("https://www.researchgate.net/publication/284825989_Evaluation_of_carbon_dioxide-nitrogen_separation_through_fixed_bed_measurements_and_simulations")

https://www.researchgate.net/profile/Rafael_Rios8
https://www.researchgate.net/profile/Leolincoln_Da_Silva_Correia
https://www.researchgate.net/profile/Moises_Bastos-Neto
https://www.researchgate.net/profile/Antonio_Torres10
